In [1]:
from environments.environment import Environment
import os


pybullet build time: Sep 22 2020 00:56:01
Downloading...
From: https://drive.google.com/uc?id=1Cc_fDSBL6QiDvNT4dpfAEbhbALSVoWcc
To: /media/marunyu/Data/study/DELFT/master_thesis/RA/ur5e.zip
100%|██████████████████████████████████████| 2.94M/2.94M [00:00<00:00, 5.08MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yOMEm-Zp_DL3nItG9RozPeJAmeOldekX
To: /media/marunyu/Data/study/DELFT/master_thesis/RA/robotiq_2f_85.zip
100%|██████████████████████████████████████| 2.33M/2.33M [00:01<00:00, 1.76MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GsqNLhEl9dd4Mc3BM0dX3MibOI1FVWNM
To: /media/marunyu/Data/study/DELFT/master_thesis/RA/bowl.zip
100%|████████████████████████████████████████| 181k/181k [00:00<00:00, 1.19MB/s]
Archive:  ur5e.zip
   creating: ur5e/
   creating: ur5e/collision/
  inflating: ur5e/collision/base.stl  
  inflating: ur5e/collision/forearm.stl  
  inflating: ur5e/collision/shoulder.stl  
  inflating: ur5e/collision/upperarm.stl  
  inflating: ur5e/collision/

In [1]:
import clip
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)


device: cuda


In [2]:
model, preprocess = clip.load("ViT-B/32", device=device)
# model, preprocess = clip.load("RN50", device=device)